# Select a couple of simpler trips to compare

* Compare methodologies, which differ when handling more complex shapes
* But is it different even in simpler shapes?
* Start with Big Blue Bus and LA Metro

In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

from shared_utils import rt_dates, rt_utils
from segment_speed_utils.project_vars import SEGMENT_GCS

from prep_comparison import map_one_trip

analysis_date = rt_dates.DATES["sep2023"]

In [2]:
df_eric = gpd.read_parquet(f"{SEGMENT_GCS}speeds_eric_{analysis_date}.parquet")
df_tiff = gpd.read_parquet(f"{SEGMENT_GCS}speeds_tiff_{analysis_date}.parquet")
speed_df = pd.read_parquet(
    f"{SEGMENT_GCS}speeds_comparison_{analysis_date}.parquet")

## Side-by-Side Maps

In [3]:
df_eric.shape, df_tiff.shape 

((151014, 23), (155314, 24))

In [4]:
#df_tiff.route_id.unique()

In [5]:
#one_route = "96-13168"
#df_tiff[df_tiff.route_id==one_route].explore(
#    "route_id", tiles = "CartoDB Positron")

In [6]:
#df_tiff[df_tiff.route_id==one_route].trip_id.unique()

In [7]:
trips_to_try = {
    "metro_720": "10720012750651-JUNE23", #route_id: 720-13168
    "metro_901": "10901000590843-JUNE23", #route_id: 901-13168
    "metro_550": "10550001350610-JUNE23", # route_id: 550-13168
    "metro_230":  "10230000830600-JUNE23", # route_id: 230-13168
    "metro_96": "10096002510743-JUNE23", # route_id: 96-13168
    "bbb1": "908521", # route_id: 3639
    #"bbb2": "", #route_id
}

In [8]:
# map_one_trip(df_eric, trips_to_try["metro_720"])

In [9]:
# map_one_trip(df_tiff, trips_to_try["metro_720"])

In [10]:
map_one_trip(df_eric, trips_to_try["metro_901"])

In [11]:
map_one_trip(df_tiff, trips_to_try["metro_901"])

In [12]:
# map_one_trip(df_eric, trips_to_try["metro_550"])

In [13]:
# map_one_trip(df_tiff, trips_to_try["metro_550"])

In [14]:
# map_one_trip(df_eric, trips_to_try["metro_230"])

In [15]:
# map_one_trip(df_tiff, trips_to_try["metro_230"])

In [16]:
map_one_trip(df_eric, trips_to_try["metro_96"])

In [17]:
map_one_trip(df_tiff, trips_to_try["metro_96"])

In [18]:
# map_one_trip(df_eric, trips_to_try["bbb1"])

In [19]:
# map_one_trip(df_tiff, trips_to_try["bbb1"])

# Dig deeper into a specific LA Metro G Line (901) segment

* big speed difference, but...
* not actually the same segment, Tiff is stop to stop but Eric cuts an interpolated segment since distance between stops >1000m
* interesting case where Eric _slower_ than Tiff when general pattern is opposite, but due to segment mismatch maybe not the best to look at first

In [20]:
trip_segment = {'trip_id': trips_to_try['metro_901'], 'stop_sequence': 8}

In [21]:
filter_trip = filter(_.trip_id == trip_segment['trip_id'], _.stop_sequence == trip_segment['stop_sequence'])

In [22]:
df_eric >> filter_trip

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_id,stop_sequence,route_id,route_short_name,direction_id,...,meters_from_last,speed_from_last,delay_chg_sec,speed_mph,n_trips_shp,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour
109199,11377.078239,15606,Woodley Station,"LINESTRING (139447.609 -424394.891, 139508.002...",9010059_JUNE23,10901000590843-JUNE23,8.0,901-13168,None,0.0,...,377.078239,2.582728,NaN,5.777562,9,12.224986,6.458984,16.165203,2.502747,9.0


In [23]:
(df_eric >> filter_trip).explore()

In [24]:
df_tiff >> filter_trip

,schedule_gtfs_dataset_key,shape_array_key,geometry,stop_id,stop_sequence,loop_or_inlining,seg_idx,district,trip_instance_key,segment_meters,...,meters_elapsed,sec_elapsed,pct_segment,speed_mph,flag_recalculated,name,trip_id,shape_id,route_id,direction_id
17626,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,20c826dfb5a463334e8942cdb3e795d5,"LINESTRING (138216.368 -424448.224, 138216.382...",15606,8,0,25889,7,1ee881f8148d699747dbcd0859fcd7f6,1612.503377,...,1605.959777,259,0.995942,13.87078,0,LA Metro Bus Schedule,10901000590843-JUNE23,9010059_JUNE23,901-13168,0.0


In [25]:
(df_tiff >> filter_trip).explore()

# Dig deeper into a specific LA Metro 96 segment

* good simple example, same exact segment geom!
* Tiff: 2.93mph, Eric: 7.95mph (over 2x!)
* so... why?

In [26]:
trip_segment = {'trip_id': trips_to_try['metro_96'], 'stop_sequence': 36}

In [27]:
filter_trip = filter(_.trip_id == trip_segment['trip_id'], _.stop_sequence == trip_segment['stop_sequence'])

In [28]:
df_eric >> filter_trip

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_id,stop_sequence,route_id,route_short_name,direction_id,...,meters_from_last,speed_from_last,delay_chg_sec,speed_mph,n_trips_shp,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour
36414,16647.487169,18094,Riverside / Riverdale,"LINESTRING (162553.508 -434619.519, 162553.710...",960251_JUNE23,10096002510743-JUNE23,36.0,96-13168,96,1.0,...,184.912802,3.556015,-8.0,7.954807,4,10.440684,7.836958,15.634983,1.995032,1.333333


In [43]:
(df_eric >> filter_trip).explore()

In [30]:
df_tiff >> filter_trip

,schedule_gtfs_dataset_key,shape_array_key,geometry,stop_id,stop_sequence,loop_or_inlining,seg_idx,district,trip_instance_key,segment_meters,...,meters_elapsed,sec_elapsed,pct_segment,speed_mph,flag_recalculated,name,trip_id,shape_id,route_id,direction_id
70379,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,7a4a0395f4918ece6eb0c18164b8c053,"LINESTRING (162553.508 -434619.519, 162553.710...",18094,36,0,93587,7,edfe9a6af705fe4181d3b916df2fdddf,184.912802,...,181.149217,138,0.979647,2.936455,0,LA Metro Bus Schedule,10096002510743-JUNE23,960251_JUNE23,96-13168,1.0


In [31]:
# (df_tiff >> filter_trip).explore()

## How does `rt_parser` calculate _this segment_??

In [35]:
from rt_analysis import rt_parser
from tqdm.notebook import tqdm
import datetime as dt

In [33]:
pbar = tqdm()

0it [00:00, ?it/s]

In [36]:
analysis_date_dt = dt.date.fromisoformat(analysis_date)
single_trip_parser = rt_parser.OperatorDayAnalysis(182, analysis_date_dt, pbar,
                                                   debug_trip_list=[trips_to_try['metro_96']])

found vp parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/vp_182_2023-09-13.parquet
found trips parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/trips_182_2023-09-13.parquet
found stop times parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/st_182_2023-09-13.parquet
found stops parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/stops_182_2023-09-13.parquet
found shapes parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/shapes_182_2023-09-13.parquet
debug mode! 
 only including trips:
['10096002510743-JUNE23']


/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


could not interpolate segments for shape 400949_JUNE23
could not interpolate segments for shape 804WB_RC_221121
could not interpolate segments for shape 7540100_JUNE23
could not interpolate segments for shape 940256_JUNE23
could not interpolate segments for shape 2340074_JUNE23
could not interpolate segments for shape 7610074_JUNE23
could not interpolate segments for shape 9100210_JUNE23
could not interpolate segments for shape 7201296_JUNE23
could not interpolate segments for shape 940258_JUNE23
could not interpolate segments for shape 805EB_190513
could not interpolate segments for shape 330574_JUNE23
could not interpolate segments for shape 801SB_RC_221121
could not interpolate segments for shape 8570001_JUNE23
could not interpolate segments for shape 6030027_JUNE23
could not interpolate segments for shape 300816_JUNE23
could not interpolate segments for shape 2040125_JUNE23
could not interpolate segments for shape 2170317_JUNE23
could not interpolate segments for shape 6050045_JUNE

In [41]:
interpolator = single_trip_parser.position_interpolators['10096002510743-JUNE23']['rt']
interpolator

In [42]:
interpolator.detailed_speed_map()

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_parser.py:153: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.dissolve().centroid


In [45]:
single_trip_parser.trs >> arrange(_.shape_meters)

,stop_id,stop_name,geometry,shape_id,shape_meters
30,30003,Burbank Station,POINT (155506.776 -424893.606),960251_JUNE23,0.000000
3,13101,Ikea / Verdugo,POINT (156022.412 -425049.524),960251_JUNE23,774.975303
19,6614,Ikea / Angeleno,POINT (155894.931 -424942.170),960251_JUNE23,941.630049
0,4642,1st / Olive,POINT (155787.560 -424850.766),960251_JUNE23,1082.611930
36,8611,Olive / San Fernando,POINT (155881.823 -424739.677),960251_JUNE23,1257.395874
32,10122,Olive / Glenoaks,POINT (156105.535 -424474.922),960251_JUNE23,1604.053106
5,13548,Glenoaks / Verdugo,POINT (156364.460 -424684.782),960251_JUNE23,1972.315904
26,13546,Glenoaks / Providencia,POINT (156577.258 -424876.708),960251_JUNE23,2264.294368
22,13545,Glenoaks / Alameda,POINT (156895.979 -425160.267),960251_JUNE23,2687.940795
38,18009,Alameda / San Fernando,POINT (156676.941 -425428.286),960251_JUNE23,3069.639235


In [52]:
single_trip_parser.trs >> filter(_.stop_id.isin(['18093', '18094'])) >> arrange(_.shape_meters)

,stop_id,stop_name,geometry,shape_id,shape_meters
6,18093,Riverside / Dorris,POINT (162545.816 -434626.873),960251_JUNE23,16462.574366
25,18094,Riverside / Riverdale,POINT (162680.185 -434755.188),960251_JUNE23,16647.487169


In [49]:
interpolator.cleaned_positions >> arrange(_.shape_meters) >> head(3)

,gtfs_dataset_name,schedule_gtfs_dataset_key,trip_instance_key,location_timestamp_local,gtfs_dataset_key_x,geometry,trip_key,gtfs_dataset_key_y,route_long_name,route_desc,shape_meters,secs_from_last,meters_from_last,speed_from_last
170,LA Metro Bus Vehicle Positions,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,edfe9a6af705fe4181d3b916df2fdddf,2023-09-13 07:36:16,65d9589130415c685b89f4f7c2d8bd7e,POINT (155549.915 -424939.886),1931ca6ce7320c0ace6fa3684d27897a,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,Metro Local Line,DOWNTOWN LA- BURBANK STA VIA GRIFFITH PARK,62.810513,NaN,NaN,NaN
192,LA Metro Bus Vehicle Positions,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,edfe9a6af705fe4181d3b916df2fdddf,2023-09-13 07:36:37,65d9589130415c685b89f4f7c2d8bd7e,POINT (155551.794 -424941.849),1931ca6ce7320c0ace6fa3684d27897a,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,Metro Local Line,DOWNTOWN LA- BURBANK STA VIA GRIFFITH PARK,65.518237,21.0,2.707724,0.128939
70,LA Metro Bus Vehicle Positions,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,edfe9a6af705fe4181d3b916df2fdddf,2023-09-13 07:37:16,65d9589130415c685b89f4f7c2d8bd7e,POINT (155551.798 -424942.071),1931ca6ce7320c0ace6fa3684d27897a,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,Metro Local Line,DOWNTOWN LA- BURBANK STA VIA GRIFFITH PARK,65.667715,39.0,0.149478,0.003833


In [50]:
(interpolator.cleaned_positions >> arrange(_.shape_meters)).explore()

TypeError: Object of type Timestamp is not JSON serializable

## How does `rt_segment_speeds` calculate _this segment_??